In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, json
from astropy import constants as const
from astropy import units as u

# increase label text size to 14
font = {'size': 14}
plt.rc('font', **font)

### Load data

In [46]:
data = pd.read_csv("materials/neutron_att_coeff.csv", sep=',')
print(data)

       material  molar_mass  density  scattering  absorption_2200
0        copper     63.5460    8.960       8.030            3.780
1          lead    207.2000   11.350      11.118            0.171
2         water     18.0153    0.997     168.000         1365.000
3  polyethylene     14.0266    0.960      -1.000           -1.000


### Input parameters

In [47]:
# Material
material = "lead"

# Energy of incoming radiation
neutron_energy = 25.30e-3 * u.eV    # 25.30 meV (2200m/s)
gamma_energy = 2.615e6 * u.eV       # Tl-208 peak @ 2.615 MeV

# Max count rate allowed
max_count_rate = 0.1 / u.yr         # 1 count per year

# Background radiation levels
incoming_neutron_flux = 1.51e-7 / u.cm**2 / u.s 
incoming_gamma_counts = 0.013 / u.s

# Surface area of detector
surface_area = 5184 * u.cm**2

print("===== Input parameters =====")
print(f"Max count rate: {max_count_rate.to(1/u.s)} = {max_count_rate}")
print(f"Neutron energy: {neutron_energy.to(u.MeV)} = {neutron_energy.to(u.eV)}")
print(f"Gamma energy: {gamma_energy.to(u.MeV)} = {gamma_energy.to(u.eV)}")
print(f"Incoming neutron flux: {incoming_neutron_flux}")
print(f"Incoming gamma counts: {incoming_gamma_counts}")
print(f"Surface area: {surface_area}")
print(f"Material: {material}")

===== Input parameters =====
Max count rate: 3.168808781402895e-09 1 / s = 0.1 1 / yr
Neutron energy: 2.5299999999999998e-08 MeV = 0.0253 eV
Gamma energy: 2.6149999999999998 MeV = 2615000.0 eV
Incoming neutron flux: 1.51e-07 1 / (cm2 s)
Incoming gamma counts: 0.013 1 / s
Surface area: 5184.0 cm2
Material: lead


In [48]:
# Get material data
material_data = data[data['material'] == material]

density     = material_data['density'].values[0] * u.g / u.cm**3
molar_mass  = material_data['molar_mass'].values[0] * u.g / u.mol
scat_coeff  = material_data['scattering'].values[0] * u.barn
abs_coeff   = material_data['absorption_2200'].values[0] * u.barn

# Calculate number density
number_density = (density * const.N_A / molar_mass).to(1/u.cm**3)
print(f"===== Material: {material} =====")
print(f"Number density: {number_density}")

# Calculate reduction required
incoming_counts = (incoming_neutron_flux * surface_area).to(1/u.yr)
reduction_required = (max_count_rate / incoming_counts).to(u.dimensionless_unscaled)
print(f"Incoming counts: {incoming_counts.to(1/u.s)} = {incoming_counts}")
print(f"Reduction required: {reduction_required}")
print()

# Scale neutron absorption coefficient
velocity = np.sqrt(2 * neutron_energy / const.m_n).to(u.m/u.s)
scale_factor = ((2200 * u.m / u.s) / velocity).to(u.dimensionless_unscaled) # scales with 1/v
abs_coeff = scale_factor * abs_coeff
print(f"Neutron velocity: {velocity}")
print(f"Absorption coefficient scale factor: {scale_factor}")
print(f"Scaled absorption coefficient: {scat_coeff}")
print(f"Scattering coefficient: {abs_coeff}")

# Calculate total neutron attenuation coefficient
total_coeff = (scat_coeff + abs_coeff).to(u.cm**2)
print(f"Total attenuation coefficient: {total_coeff}")
print()

# Calculate thickness required
thickness_scat = (-(np.log(reduction_required))/(scat_coeff * number_density)).to(u.cm)
thickness_abs = (-(np.log(reduction_required))/(abs_coeff * number_density)).to(u.cm)
thickness = (-(np.log(reduction_required))/(total_coeff * number_density)).to(u.cm)
print(f"Thickness required (scattering only): {thickness_scat}")
print(f"Thickness required (absorption only): {thickness_abs}")
print(f"Thickness required: {thickness}")

===== Material: lead =====
Number density: 3.2988078004826253e+22 1 / cm3
Incoming counts: 0.0007827839999999999 1 / s = 24702.7843584 1 / yr
Reduction required: 4.048126662531293e-06

Neutron velocity: 2200.049482932255 m / s
Absorption coefficient scale factor: 0.9999775082639554
Scaled absorption coefficient: 0.17099615391313638 barn
Scattering coefficient: 0.17099615391313638 barn
Total attenuation coefficient: 1.1288996153913136e-23 cm2

Thickness required (scattering only): 33.85649253414646 cm
Thickness required (absorption only): 2201.31549967992 cm
Thickness required: 33.343663055829985 cm
